# Navier Stokes Equatoin

In [1]:
import numpy as np
import time
import scipy.sparse.linalg as sp_la

# --- Core imports ---
from pycutfem.core.mesh import Mesh
from pycutfem.core.dofhandler import DofHandler
from pycutfem.utils.meshgen import structured_quad

# --- UFL-like imports ---
from ufl.functionspace import FunctionSpace
from ufl.expressions import (
    TrialFunction, TestFunction, VectorTrialFunction, VectorTestFunction,
    Function, VectorFunction, Constant, grad, inner, dot, div
)
from ufl.measures import dx
from ufl.forms import BoundaryCondition, assemble_form

# 1. ============================================================================
#    SETUP (Meshes, DofHandler, BCs - same as before)
# ===============================================================================
L, H = 1.0, 1.0
NX, NY = 4, 4
nodes_q2, elems_q2, _, corners_q2 = structured_quad(L, H, nx=NX, ny=NY, poly_order=2)
mesh_q2 = Mesh(nodes=nodes_q2, element_connectivity=elems_q2, elements_corner_nodes=corners_q2, element_type="quad", poly_order=2)

nodes_q1, elems_q1, _, corners_q1 = structured_quad(L, H, nx=NX, ny=NY, poly_order=1)
mesh_q1 = Mesh(nodes=nodes_q1, element_connectivity=elems_q1, elements_corner_nodes=corners_q1, element_type="quad", poly_order=1)

fe_map = {'ux': mesh_q2, 'uy': mesh_q2, 'p': mesh_q1}
dof_handler = DofHandler(fe_map, method='cg')

bc_tags = {'bottom_wall': lambda x,y: np.isclose(y,0), 'left_wall': lambda x,y: np.isclose(x,0), 'right_wall': lambda x,y: np.isclose(x,L), 'top_lid': lambda x,y: np.isclose(y,H)}
mesh_q2.tag_boundary_edges(bc_tags)
mesh_q1.nodes_list[0].tag = 'pressure_pin_point'

class DataBC:
    Um = 1.5
    H = H
bcs = [
    BoundaryCondition('ux', 'dirichlet', 'left_wall',   lambda x, y: 4*DataBC.Um*y*(DataBC.H-y)/(DataBC.H**2)),
    BoundaryCondition('uy', 'dirichlet', 'left_wall',   lambda x, y: 0.0),
    BoundaryCondition('ux', 'dirichlet', 'right_wall',  lambda x, y: 0.0),
    BoundaryCondition('uy', 'dirichlet', 'right_wall',  lambda x, y: 0.0),
    BoundaryCondition('ux', 'dirichlet', 'bottom_wall', lambda x, y: 0.0),
    BoundaryCondition('uy', 'dirichlet', 'bottom_wall', lambda x, y: 0.0),
    BoundaryCondition('ux', 'dirichlet', 'top_lid',     lambda x, y: 0.0),
    BoundaryCondition('uy', 'dirichlet', 'top_lid',     lambda x, y: 0.0),
    BoundaryCondition('p', 'dirichlet', 'pressure_pin_point', lambda x, y: 0.0)
]


In [2]:
# --- Define Constants and Function Spaces ---
# --- Define Constants and Function Spaces ---
rho = Constant(1.0)
dt = Constant(0.1)
theta = Constant(0.5)
mu = Constant(1.0e-2)

velocity_space = FunctionSpace("velocity", ['ux', 'uy'])
pressure_space = FunctionSpace("pressure", ['p'])

# --- Define Functions ---
# Trial functions (the unknowns of the Jacobian system, i.e., the corrections)
du = VectorTrialFunction(velocity_space) # Correction for velocity
dp = TrialFunction(pressure_space.field_names[0]) # Correction for pressure

# Test functions
v = VectorTestFunction(velocity_space)
q = TestFunction(pressure_space.field_names[0])

# Functions to hold solutions at different time steps/iterations
u_k = VectorFunction(name="u_k", field_names=['ux', 'uy']) # Solution at current Newton iter k
u_n = VectorFunction(name="u_n", field_names=['ux', 'uy']) # Solution at previous time step n
p_k = Function(name="p_k", field_name='p')

a_form = (
    # Time derivative term: d/dt(u) -> du/dt
    rho * dot(du, v) / dt * dx() +
    
    # Convection term: theta * [ ((du ⋅ ∇)u_k) ⋅ v + ((u_k ⋅ ∇)du) ⋅ v ]
    theta * rho * dot(dot(du, grad(u_k)), v) * dx() +
    theta * rho * dot(dot(u_k, grad(du)), v) * dx() +

    # Diffusion term: theta * mu * (∇du : ∇v)
    theta * mu * inner(grad(du), grad(v)) * dx() -
    
    # Pressure term: -theta * dp * (∇⋅v)
    theta * dp * div(v) * dx() +
    
    # Continuity term: q * (∇⋅du)
    q * div(du) * dx()
)

L_form = (
    # Time derivative: -(u_k - u_n)/dt ⋅ v
    -rho * dot(u_k - u_n, v) / dt * dx() -

    # Convection terms at theta and (1-theta)
    (theta * rho * dot(dot(u_k, grad(u_k)), v) * dx()) -
    ((1.0 - theta) * rho * dot(dot(u_n, grad(u_n)), v) * dx()) -
    
    # Diffusion terms at theta and (1-theta)
    (theta * mu * inner(grad(u_k), grad(v)) * dx()) -
    ((1.0 - theta) * mu * inner(grad(u_n), grad(v)) * dx()) +
    
    # Pressure terms
    p_k * div(v) * dx() -
    
    # Continuity term
    q * div(u_k) * dx() 
    
)

equation = a_form == L_form


In [3]:
# 3. ============================================================================
#    NEWTON'S LOOP
# ===============================================================================

# --- Simulation Parameters ---
T_end = 2.0
dt_val = dt.value
num_steps = int(T_end / dt_val)
newton_tol = 1e-6
max_newton_iter = 20

# Create a second list of Boundary Conditions for the homogeneous case
bcs_homog = [
    BoundaryCondition(bc.field, bc.method, bc.domain_tag, lambda x, y: 0.0) 
    for bc in bcs
]

# Initialize solution vectors
U_k = np.zeros(dof_handler.total_dofs)
U_n = np.zeros(dof_handler.total_dofs)

# Set initial condition (at t=0) by applying non-homogeneous BCs to U_n
dirichlet_dofs_map = dof_handler.get_dirichlet_data(bcs)
dirichlet_dofs = np.array(list(dirichlet_dofs_map.keys()))
dirichlet_values = np.array(list(dirichlet_dofs_map.values()))
U_n[dirichlet_dofs] = dirichlet_values

# Start time-stepping
print("--- Starting time integration with Manual UFL Forms ---")
start_time = time.time()
for n in range(num_steps):
    current_time = (n + 1) * dt_val
    print(f"\n--- Time step {n+1}/{num_steps} | t = {current_time:.2f}s ---")

    # Update previous step solution U_n for use in the Residual
    # (The compiler will get this from the context)
    
    # Initial guess for Newton's is the solution from the previous step
    U_k[:] = U_n

    # Start Newton's iterations
    for k in range(max_newton_iter):
        # Pass both U_k and U_n to the assembler context
        solution_vectors = {'u_k': U_k, 'u_n': U_n}
        A, b = assemble_form(equation, 
                             dof_handler=dof_handler, 
                             bcs=bcs_homog,
                             quad_order=5,
                             solution_vector=solution_vectors)

        # Calculate the L2 norm of the residual on the free DoFs
        free_dofs = np.ones(dof_handler.total_dofs, dtype=bool)
        free_dofs[dirichlet_dofs] = False
        res_norm = np.linalg.norm(b[free_dofs])
        
        print(f"  Newton Iter: {k+1} | Residual norm: {res_norm:.4e}")

        if res_norm < newton_tol:
            print(f"  Newton's method converged in {k+1} iterations.")
            break

        # Solve A * delta_U = b for the correction
        delta_U = sp_la.spsolve(A, b)

        # Apply the correction to the solution
        U_k += delta_U
        
        # Re-apply non-homogeneous BCs to the solution vector
        # This is good practice for stability.
        U_k[dirichlet_dofs] = dirichlet_values

    else: 
        print("  Warning: Newton's method did not converge!")
        break 

    # Update U_n for the next time step
    U_n[:] = U_k

end_time = time.time()
print(f"\n--- Simulation finished in {end_time - start_time:.2f} seconds ---")

--- Starting time integration with Manual UFL Forms ---

--- Time step 1/20 | t = 0.10s ---


ValueError: operands could not be broadcast together with shapes (18,18) (9,9) (18,18) 